In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/Shareddrives/DL2-project/tfx

/content/drive/Shareddrives/DL2-project/tfx


In [4]:
# !pip install google-auth>=1.21.1
# !pip install httplib2==0.17.0
# !pip install tflite-support
# !pip install -U tfx
# !pip install python-snappy

In [5]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx

from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform

from tfx.dsl.experimental import latest_blessed_model_resolver

from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.utils.dsl_utils import external_input

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [6]:
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen

In [7]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.0
TFX version: 0.25.0


In [8]:
_pipeline_name = 'plantpatho_native_keras'

# root directory for TFX pip package installation.
_plantpatho_root = "."
print("Root dir: " + _plantpatho_root)

_data_root = os.path.join(_plantpatho_root, 'data')

# Serving path
_serving_model_dir = os.path.join('./serving_model')
print("TF Serving dir: ", _serving_model_dir)

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

# Set up utils file
_module_file = os.path.join(_plantpatho_root, 'plant_utils_native_keras.py')

input_config = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name='train', pattern='train/*'),
    example_gen_pb2.Input.Split(name='eval', pattern='test/*')
])

_tfx_root = os.path.join(".", 'tfx_output')

_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)

metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name, 'metadata.db')

_labels_path = os.path.join(_data_root, 'labels.txt')

Root dir: .
TF Serving dir:  ./serving_model


In [9]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

# ExampleGen

In [10]:
example_gen = ImportExampleGen(input_base="./data/", input_config=input_config)

In [11]:
example_gen

ImportExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7f9b626431d0>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f9b62643940>, driver_class: <class 'tfx.components.example_gen.driver.Driver'>, component_id: ImportExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
)})

In [12]:
context.run(example_gen)

INFO:absl:Running driver for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for ImportExampleGen
INFO:absl:Generating examples.


INFO:absl:Reading input TFRecord data ./data/train/*.
INFO:absl:Reading input TFRecord data ./data/test/*.
INFO:absl:Examples generated.
INFO:absl:Running publisher for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:323193155,xor_checksum:1607670695,sum_checksum:1607670695\nsplit:eval,num_files:1,total_bytes:78770838,xor_checksum:1607670693,sum_checksum:1607670693"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [13]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/ImportExampleGen/examples/1


In [14]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

In [15]:
train_uri

'/tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/ImportExampleGen/examples/1/train'

In [16]:
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

In [17]:
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [18]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [19]:
# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# StatisticsGen

In [20]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [21]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Data statistics are a little lackluster for this image dataset

#SchemaGen

In [22]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [23]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'image',BYTES,required,single,-
'label',INT,required,single,-


# Example Validator

In [24]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/ExampleValidator/anomalies/4/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/ExampleValidator/anomalies/4/eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [25]:
context.show(example_validator.outputs['anomalies'])

# Transform

In [26]:
# Performs transformations and feature engineering in training and serving.
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_module_file)

context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.


output tensor: {'image_xf': <tf.Tensor 'sub:0' shape=(None, 224, 224, 3) dtype=float32>, 'label_xf': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f9b101e6be0>}
Instructions for updating:
Use ref() instead.


INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.


...

output tensor: {'image_xf': <tf.Tensor 'sub:0' shape=(None, 224, 224, 3) dtype=float32>, 'label_xf': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f9b1015f748>}
output tensor: {'image_xf': <tf.Tensor 'sub:0' shape=(None, 224, 224, 3) dtype=float32>, 'label_xf': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f9b10130160>}
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-12-11T21_00_41.715880-k6xgfrtc/Transform/transform_graph/5/.temp_path/tftransform_tmp/a1a1c59dbb78406fb9bf7080997db456/saved_model.pb


INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature image has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature label has no shape. Setting to VarLenSparseTensor.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ValueError: ignored

In [ ]:
transform.outputs

In [ ]:
train_uri = transform.outputs['transform_graph'].get()

#Trainer

In [ ]:
# trainer = Trainer(
#       module_file=_module_file,
#       custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
#       examples=transform.outputs['transformed_examples'],
#       transform_graph=transform.outputs['transform_graph'],
#       schema=schema_gen.outputs['schema'],
#       train_args=trainer_pb2.TrainArgs(num_steps=160),
#       eval_args=trainer_pb2.EvalArgs(num_steps=4),
#       custom_config={'labels_path': labels_path})